# Food delivery data preprocess

## Import packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [7]:
data = pd.read_csv('../Data/food_delivery_data.csv')
data.columns = ['store_id', 'promised delivery time', 'ordering time', 'latitude', 'longitude']
data.head()

,store_id,promised delivery time,ordering time,latitude,longitude
0,1,2019-01-01 07:30:00,2019-01-01 06:30:07,31.167880,121.536578
1,1,2019-01-01 07:30:00,2019-01-01 06:35:02,31.175481,121.537246
2,1,2019-01-01 08:00:00,2019-01-01 06:47:55,31.179777,121.553969
3,1,2019-01-01 08:00:00,2019-01-01 06:48:59,31.175272,121.540357
4,1,2019-01-01 07:45:00,2019-01-01 06:50:00,31.175667,121.548509


## Add new columns

In [8]:
# create a new column ('delivery time (second)')
import datetime

data['delivery time (s)'] = 0

for index, row in data.iterrows():
    t1 = datetime.datetime.strptime(data['ordering time'][index], '%Y-%m-%d %H:%M:%S')
    t2 = datetime.datetime.strptime(data['promised delivery time'][index], '%Y-%m-%d %H:%M:%S')
    diff = int((t2 - t1).total_seconds())
    data.at[index, 'delivery time (s)'] = diff

# create a new column ('delivery time (minute)')
data['ordering time'] = pd.to_datetime(data['ordering time'])
data['promised delivery time'] = pd.to_datetime(data['promised delivery time'])
data['delivery time (m)'] = (data['promised delivery time'] - data['ordering time']).astype('timedelta64[m]')

data.head()

,store_id,promised delivery time,ordering time,latitude,longitude,delivery time (s),delivery time (m)
0,1,2019-01-01 07:30:00,2019-01-01 06:30:07,31.167880,121.536578,3593,59.0
1,1,2019-01-01 07:30:00,2019-01-01 06:35:02,31.175481,121.537246,3298,54.0
2,1,2019-01-01 08:00:00,2019-01-01 06:47:55,31.179777,121.553969,4325,72.0
3,1,2019-01-01 08:00:00,2019-01-01 06:48:59,31.175272,121.540357,4261,71.0
4,1,2019-01-01 07:45:00,2019-01-01 06:50:00,31.175667,121.548509,3300,55.0


In [9]:
from geopy import distance

def distance_loc(p1, p2):
    '''
    Input: locations (latitude, longitude)
     - p1: location 1
     - p2: location 2
    Output: distance in km
    '''
    return distance.distance(p1, p2).km

In [17]:
# get the boundary of the restaurants
restaurants = list(data['store_id'].unique())
restaurants_dict = {k:[] for k in restaurants}
boundary_dict = {k:[] for k in restaurants}
data['distance (m)'] = 0.0

for r in restaurants:
    df = data.loc[data['store_id'] == r]

    # get boundary
    west = df.longitude.min()
    east = df.longitude.max()
    south = df.latitude.min()
    north = df.latitude.max()
    boundary_dict[r] = [west, east, south, north]

    # restaurant location
    r_longi = np.average([west, east])
    r_latit = np.average([south, north])
    restaurants_dict[r] = (r_latit, r_longi)

restaurants_dict


{1: (31.1839785, 121.5382955), 2: (22.496323, 113.937388)}

In [19]:
for index, row in data.iterrows():
    r = row['store_id']
    data.at[index, 'distance (m)'] = distance_loc((row['latitude'], row['longitude']), restaurants_dict[r]) * 1000

data.head()

,store_id,promised delivery time,ordering time,latitude,longitude,delivery time (s),delivery time (m),distance,distance (m)
0,1,2019-01-01 07:30:00,2019-01-01 06:30:07,31.167880,121.536578,3593,59.0,1.792374,1792.374399
1,1,2019-01-01 07:30:00,2019-01-01 06:35:02,31.175481,121.537246,3298,54.0,0.947436,947.436336
2,1,2019-01-01 08:00:00,2019-01-01 06:47:55,31.179777,121.553969,4325,72.0,1.564975,1564.975000
3,1,2019-01-01 08:00:00,2019-01-01 06:48:59,31.175272,121.540357,4261,71.0,0.985111,985.111196
4,1,2019-01-01 07:45:00,2019-01-01 06:50:00,31.175667,121.548509,3300,55.0,1.340554,1340.554487


In [28]:
corr = data['delivery time (s)'].corr(data['distance (m)'])
corr

0.051988352112812816

## Write data to file

In [26]:
# get date
dates = [i.strftime('%Y-%m-%d') for i in data['promised delivery time'].dt.date.unique()]
print(dates)

for r in restaurants:
    # for each restaurant
    df = data.loc[data['store_id'] == r]
    df.to_csv(f'../Data/restaurant_{r}.csv')
    print(f'Finish writing data with restaurant {r}.')

    # for each date
    for d in dates:
        df_time = df.loc[df['promised delivery time'].dt.date == datetime.datetime.strptime(d,'%Y-%m-%d').date()]
        df_time.to_csv(f'../Data/restaurant_{r}_{d}.csv')
        print(f'Finish writing data with restaurant {r} in day {d} to csv file.')

print('Finish data writing!')

['2019-01-01', '2019-01-02', '2019-01-03', '2019-01-04', '2019-01-05', '2018-12-09', '2018-12-10', '2018-12-11', '2018-12-12', '2018-12-13', '2018-12-14', '2018-12-15', '2018-12-30', '2018-12-31', '2019-01-06', '2019-01-07', '2019-01-08', '2019-01-09', '2019-01-10', '2019-01-11', '2019-01-12']
Finish writing data with restaurant 1.
Finish writing data with restaurant 1 in day 2019-01-01 to csv file.
Finish writing data with restaurant 1 in day 2019-01-02 to csv file.
Finish writing data with restaurant 1 in day 2019-01-03 to csv file.
Finish writing data with restaurant 1 in day 2019-01-04 to csv file.
Finish writing data with restaurant 1 in day 2019-01-05 to csv file.
Finish writing data with restaurant 1 in day 2018-12-09 to csv file.
Finish writing data with restaurant 1 in day 2018-12-10 to csv file.
Finish writing data with restaurant 1 in day 2018-12-11 to csv file.
Finish writing data with restaurant 1 in day 2018-12-12 to csv file.
Finish writing data with restaurant 1 in day 